# Unit 5 - Financial Planning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install colab-env --upgrade

  Created wheel for colab-env: filename=colab_env-0.2.0-cp37-none-any.whl size=3837 sha256=626f55f0bdd442e8397f761f1126aa415111669eff108745a20c3c30d9a9f34f
  Stored in directory: /root/.cache/pip/wheels/81/9c/85/7ec1faca43d1a5df32f33f4e22d5ae91f301a5406779528ead
Successfully built colab-env


In [3]:
!pip install alpaca-trade-api

     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 71kB 4.3MB/s 


In [ ]:
#The most reliable way to import local resources

In [16]:
!cp '/content/drive/MyDrive/BootCampSpot/MCForecastTools.py' .
!cp '/content/drive/MyDrive/BootCampSpot/pandasPalmer.py' .
!cp '/content/drive/MyDrive/BootCampSpot/rsaPalmer.py' .
!cp '/content/drive/MyDrive/BootCampSpot/.env' .

In [12]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import alpaca_trade_api as tradeapi
import importlib
#import MCForecastTools
from MCForecastTools import MCSimulation
import rsaPalmer

%matplotlib inline

In [17]:

importlib.reload(rsaPalmer)

Default Path = /content/drive/MyDrive/BootCampSpot/
Default Env file Path = /content/drive/MyDrive/BootCampSpot/.env exists
File rsaPalmer is being imported


<module 'rsaPalmer' from '/content/rsaPalmer.py'>

In [20]:
# Load .env enviroment variables
# Loads the .env file and set's the ENV variables for use.
env_val = os.getenv('PalmerEnv')
if (env_val == None):
  env_file = '/content/drive/MyDrive/BootCampSpot/.env'
  if not load_dotenv(find_dotenv(filename=env_file, raise_error_if_not_found=True)):
    print('Failed to load the environment variables, ensure the Directory exists')
  else:
    print('Sucessfully Loaded the Environment Variables')
elif env_val == "True":
  print('The Environmental Variables are already loaded')


The Environmental Variables are already loaded


## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [22]:
import requests

In [23]:
# Set current amount of crypto assets
#@title My BitCoin and Etherium
my_btc        =  1.2 #@param {type:"number"}
my_eth        =  5.6 #@param {type:"number"}
#avg_hh_income = 12000 #@param {type:"number"}

In [24]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [25]:
# Fetch current BTC price
btc_info = requests.get(btc_url)

if (btc_info.status_code==200):
  btc_price = btc_info.json()["data"]["1"]["quotes"]["CAD"]["price"]
else:
  print(f'Failed to connect to [{btc_url}] for the current price')

# Fetch current ETH price
eth_info = requests.get(eth_url)

if (eth_info.status_code==200):
  eth_price = eth_info.json()["data"]["1027"]["quotes"]["CAD"]["price"]
else:
  print(f'Failed to connect to [{eth_url}] for the current price')

# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is CAD${my_btc_value:0,.2f}")
print(f"The current value of your {my_eth} ETH is CAD${my_eth_value:0,.2f}")


The current value of your 1.2 BTC is CAD$58,923.18
The current value of your 5.6 ETH is CAD$17,696.53


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [35]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [38]:
# Set Alpaca API key and secret
# The keys that end with '_enc' have been encrypted in the .env file, it will be automatically decrypted.
alpc_API_key = rsaPalmer.Get_API_Key('ALPACA_API_KEY_enc')[1] # Gets and decrypts key
alpc_SECRET_key = rsaPalmer.Get_API_Key('ALPACA_SECRET_KEY_enc')[1] # Gets and decrypts key

# Create the Alpaca API object
trial_market = 'https://paper-api.alpaca.markets'
import alpaca_trade_api as alpaca
api = alpaca.REST(alpc_API_key, alpc_SECRET_key, trial_market, api_version="v2")

Public file exists
Private file exists
Public file exists
Private file exists


In [42]:
# Format current date as ISO format
from datetime import datetime
current_date = pd.Timestamp(datetime.now()).isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
df_ticker = api.get_barset(
    symbols=tickers,
    timeframe=timeframe,
    start=current_date,
    limit= 1000
).df

# Preview DataFrame
df_ticker.head()

AGG            ...     SPY            
                             open      high  ...   close      volume
time                                         ...                    
2017-06-22 00:00:00-04:00  110.06  110.0947  ...     NaN         NaN
2017-06-23 00:00:00-04:00  110.01  110.1085  ...  243.16  45792641.0
2017-06-26 00:00:00-04:00  110.19  110.2400  ...  243.27  43153677.0
2017-06-27 00:00:00-04:00  109.93  109.9800  ...  241.36  60858189.0
2017-06-28 00:00:00-04:00  109.85  109.8800  ...  243.46  50076355.0

[5 rows x 10 columns]

In [ ]:
df_ticker.info()

In [ ]:
df_AGG_close = df_ticker['AGG']['close']
df_AGG_close

In [107]:
# Pick AGG and SPY close prices
df_AGG_close = df_ticker['AGG']['close']
df_SPY_close = df_ticker['SPY']['close']
df_COMB_close = pd.concat(objs=[df_AGG_close,df_SPY_close], axis='columns', join='inner')
df_COMB_close.columns = tickers

# Last Trading Day
last_trading_day = max(df_COMB_close.index)

date_filtr = (df_COMB_close.index == last_trading_day)

agg_close_price = df_COMB_close.loc[date_filtr, 'AGG'].values[0]
spy_close_price = df_COMB_close.loc[date_filtr, 'SPY'].values[0]

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")


Current AGG closing price: $114.97
Current SPY closing price: $425.29


In [116]:
# Compute the current value of shares
my_agg_value = agg_close_price * my_agg
my_spy_value = spy_close_price * my_spy

# Print current value of shares
print(f"The current value of your {my_spy:3} SPY shares is ${my_spy_value:10,.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:10,.2f}")

The current value of your  50 SPY shares is $ 21,264.50
The current value of your 200 AGG shares is $ 22,994.00


### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
# YOUR CODE HERE!

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")